In [2]:
import pandas as pd
import numpy as np
import sklearn.preprocessing as skpp
# XGBoost
# https://xgboost.readthedocs.io/en/stable/python/python_api.html#module-xgboost.sklearn
import xgboost as xgb # must be in AMN_windows env
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_predict as cvp
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.multioutput import MultiOutputRegressor
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from typing import Dict, Tuple, List


PATH = "biolog_data.txt"
all_data = pd.read_csv(PATH, comment="<", encoding='cp1252', delimiter="\t")
# all_data_norm = (all_data - all_data.mean()) - all_data.std()
all_data

,Experiment Description,Data Definition,Well Substrate,Data Value,Data Submitter
0,b3066 mutant Biolog assay,OD600 growth,(-)Shikimic Acid,1.74,Blattner Laboratory
1,b3072 mutant Biolog assay,OD600 growth,(-)Shikimic Acid,0.89,Blattner Laboratory
2,b3076 mutant Biolog assay,OD600 growth,(-)Shikimic Acid,1.07,Blattner Laboratory
3,b3091 mutant Biolog assay,OD600 growth,(-)Shikimic Acid,0.78,Blattner Laboratory
4,b3093 mutant Biolog assay,OD600 growth,(-)Shikimic Acid,1.43,Blattner Laboratory
...,...,...,...,...,...
100907,b4129 mutant Biolog assay,OD600 growth,Xylitol,0.19,Blattner Laboratory
100908,b4138 mutant Biolog assay,OD600 growth,Xylitol,0.22,Blattner Laboratory
100909,b4173 mutant Biolog assay,OD600 growth,Xylitol,0.15,Blattner Laboratory
100910,b4174 mutant Biolog assay,OD600 growth,Xylitol,0.15,Blattner Laboratory


In [4]:
print("Unique mutants number: ", len(set(all_data['Experiment Description'])))
print("Unique substrates number: ", len(set(all_data['Well Substrate'])))

Unique mutants number:  212
Unique substrates number:  390


82680

In [18]:
target_gene_encoder = skpp.OneHotEncoder()
which_CS_encoder = skpp.OneHotEncoder()

raw_target_gene = all_data["Experiment Description"].values.reshape(-1, 1)
raw_which_medium = all_data["Well Substrate"].values.reshape(-1, 1)

target_gene_encoder.fit(raw_target_gene)
which_CS_encoder.fit(raw_which_medium)

X_left = target_gene_encoder.transform(raw_target_gene).toarray()
X_right = which_CS_encoder.transform(raw_which_medium).toarray()

X = np.hstack((X_left, X_right))

# raw_X = all_data.iloc[:,:3].values
# enc.fit(raw_X)
# X = enc.transform(raw_X).toarray()

Y = all_data["Data Value"].values.reshape(-1, 1)
print(X.shape, Y.shape)

(100912, 602) (100912, 1)


In [19]:
LOO_split = int(Y.shape[0])

xgb_preds = []

for i in range(5):

    # XGBoost = xgb.XGBRegressor(n_estimators=600, max_depth=2, learning_rate=0.01, objective='reg:squarederror', random_state=i)
    multioutputregressor = MultiOutputRegressor(xgb.XGBRegressor(objective='reg:squarederror'))

    xgb_pred = cvp(multioutputregressor, X, Y, cv=KFold(n_splits=10, shuffle=True, random_state=i), n_jobs = 5)
    
    xgb_preds.append(xgb_pred)

In [22]:
mean_xgb_pred = np.array(xgb_preds).mean(axis=0)
std_xgb = np.array(xgb_preds).std(axis=0)

print("Mean squared error between mean prediction and true fluxes", np.mean((mean_xgb_pred - Y)**2, axis=0)) # MSE between mean prediction (over 5 times 10-fold test sets) and true

print("Q²: ", r2_score(Y, mean_xgb_pred))

Mean squared error between mean prediction and true fluxes [0.19389984]
Q²:  0.525509846522625


In [9]:
xp_descriptions = all_data['Experiment Description']
genes = list(set([xp_desc.split(' ')[0] for xp_desc in xp_descriptions]))
len(genes)

212

In [10]:
TF_to_gene = pd.read_csv("NetWorkTFGene.txt", delimiter="\t", comment="#")
TF_to_gene

,1)regulatorId,2)regulatorName,3)RegulatorGeneName,4)regulatedId,5)regulatedName,6)function,7)confidenceLevel
0,ECK120011186,Fis,fis,ECK120000028,adhE,+,Strong
1,ECK120011186,Fis,fis,ECK120000040,osmE,-,Strong
2,ECK120011186,Fis,fis,ECK120000042,ansB,-,Weak
3,ECK120011186,Fis,fis,ECK120000043,apaG,+,Weak
4,ECK120011186,Fis,fis,ECK120000044,apaH,+,Weak
...,...,...,...,...,...,...,...
6055,ECK125336166,RpoN,rpoN,ECK120000323,mutM,-,Strong
6056,ECK125336166,RpoN,rpoN,ECK120000655,nth,-,Strong
6057,ECK125336166,RpoN,rpoN,ECK120002092,cspC,-,Weak
6058,ECK125336166,RpoN,rpoN,ECK120003587,yobF,-,Strong


In [11]:
import cobra
cobra_model = cobra.io.read_sbml_model("iML1515.xml")
genes_names_cobra_model = [x.id for x in cobra_model.genes]
genes_names_cobra_model

['b2551',
 'b0870',
 'b3368',
 'b2436',
 'b0008',
 'b3500',
 'b2465',
 'b0945',
 'b4467',
 'b3126',
 'b4468',
 'b2979',
 'b3916',
 'b1095',
 'b1054',
 'b1855',
 'b1260',
 'b2378',
 'b0109',
 'b1094',
 'b2836',
 'b2472',
 'b2935',
 'b1261',
 'b3426',
 'b2242',
 'b2243',
 'b2241',
 'b4054',
 'b3770',
 'b3281',
 'b1692',
 'b0243',
 'b1590',
 'b1613',
 'b0414',
 'b0894',
 'b1588',
 'b1587',
 'b0896',
 'b1589',
 'b0895',
 'b3041',
 'b0025',
 'b3553',
 'b0678',
 'b0844',
 'b0446',
 'b1478',
 'b0638',
 'b3812',
 'b1662',
 'b3399',
 'b0421',
 'b3804',
 'b0469',
 'b3997',
 'b4382',
 'b3187',
 'b3222',
 'b3225',
 'b0126',
 'b4269',
 'b0339',
 'b3942',
 'b4006',
 'b1732',
 'b0493',
 'b2508',
 'b3846',
 'b3588',
 'b0586',
 'b2341',
 'b3794',
 'b2400',
 'b3003',
 'b3993',
 'b3011',
 'b3693',
 'b2066',
 'b2901',
 'b0238',
 'b2716',
 'b3903',
 'b0125',
 'b0474',
 'b2411',
 'b3721',
 'b2518',
 'b1062',
 'b3416',
 'b3996',
 'b1281',
 'b4390',
 'b0639',
 'b0720',
 'b1398',
 'b1136',
 'b0049',
 'b2103',


In [12]:
regulator_genes_candidates = {}
corresponding_nbr_involved_reactions = []
metabolic_genes_candidates = {}

for regulator_gene in genes:
    subset = TF_to_gene.loc[TF_to_gene['2)regulatorName'] == regulator_gene]
    nbr_target_genes = len(subset)
    if nbr_target_genes > 0:
        list_regulated_genes = subset["5)regulatedName"].values
        print("According to RegulonDB, " + regulator_gene + " regulates:\n", list_regulated_genes)
        nbr_involved_reactions = 0
        for regulated_gene in list_regulated_genes:
            if regulated_gene in genes_names_cobra_model:
                print("The regulator " + regulator_gene + " targets the " + regulated_gene + " gene in the cobra model.")
                involved_reactions = cobra_model.genes[genes_names_cobra_model.index(regulated_gene)].reactions
                print([reac.name for reac in involved_reactions])
                nbr_involved_reactions += len(involved_reactions)
        regulator_genes_candidates[regulator_gene] = involved_reactions
        corresponding_nbr_involved_reactions.append(nbr_involved_reactions)
        continue

    else:
        # print(regulator_gene + " regulator has no target gene in regulonDB")
        if regulator_gene in genes_names_cobra_model:
                print("The gene " + regulator_gene + " is directly in the cobra model (it's a metabolic gene).")
                print("It involves these reactions: ")
                involved_reactions = cobra_model.genes[genes_names_cobra_model.index(regulator_gene)].reactions
                print([reac.name for reac in involved_reactions])
                metabolic_genes_candidates[regulator_gene] = involved_reactions
                continue
        else:
            print("\n\n")
            print("This gene is neither a regulator in RegulonDB, neither a gene in iML1515")
            print("\n\n")
    




This gene is neither a regulator in RegulonDB, neither a gene in iML1515



The gene b3403 is directly in the cobra model (it's a metabolic gene).
It involves these reactions: 
['Phosphoenolpyruvate carboxykinase']



This gene is neither a regulator in RegulonDB, neither a gene in iML1515



The gene b3925 is directly in the cobra model (it's a metabolic gene).
It involves these reactions: 
['Fructose-bisphosphatase']
The gene b3725 is directly in the cobra model (it's a metabolic gene).
It involves these reactions: 
['Phosphate transport via ABC system (uptake, periplasm)']
The gene b3870 is directly in the cobra model (it's a metabolic gene).
It involves these reactions: 
['Glutamine synthetase']



This gene is neither a regulator in RegulonDB, neither a gene in iML1515



The gene b3902 is directly in the cobra model (it's a metabolic gene).
It involves these reactions: 
['Rhamnulose-1-phosphate aldolase']
The gene b3517 is directly in the cobra model (it's a metabolic gene).
I

In [14]:
nbr_metabolic_genes = len(metabolic_genes_candidates) # 120 different metabolic genes
nbr_reactions_targets = sum([len(metabolic_genes_candidates[x]) for x in metabolic_genes_candidates.keys()]) # total of 271 targeted reactions
nbr_unique_reactions_targeted = len(set([next(iter(metabolic_genes_candidates[x])).name for x in metabolic_genes_candidates.keys()])) # but only 103 different reactions

print(nbr_metabolic_genes, nbr_reactions_targets, nbr_unique_reactions_targeted)

120 271 103


In [39]:
# no regulator detected because ID issue!!
len(regulator_genes_candidates)
# sum([len(regulator_genes_candidates[x]) for x in regulator_genes_candidates.keys()]) #
# len(set([next(iter(regulator_genes_candidates[x])).name for x in regulator_genes_candidates.keys()])) #

0